In [62]:
spark.stop()

In [66]:
# Notebook phân tích & xây dựng mapping cho dữ liệu bất động sản
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, to_timestamp, current_timestamp, lit, regexp_replace, trim,
    when, upper, lower, split, element_at, round as spark_round,
    avg, count, percentile_approx, stddev, min as spark_min, max as spark_max,
    udf, length, expr
)
from pyspark.sql.types import StringType, DoubleType, BooleanType
from pyspark.sql.window import Window

# Thêm thư mục gốc vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

# Tạo Spark Session
spark = SparkSession.builder \
    .appName("BatDongSan Mapping Analysis") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

print("Spark session created successfully")

Spark session created successfully


In [85]:
parquet_file = "/data/realestate/processed/silver/batdongsan/house/2025/06/12/*.parquet"
df = spark.read.parquet(parquet_file)

df_filtered = df.select(
    'location', 
    'street', 
    'street_id', 
    'ward', 
    'ward_id', 
    'district', 
    'district_id', 
    'province', 
    'province_id'
)

df_filtered.show(100, truncate=False)
df.printSchema()


+------------------------------------------------------------------------------+---------------------------------+---------+--------------------------+-------+-----------------------+-----------+--------------+-----------+
|location                                                                      |street                           |street_id|ward                      |ward_id|district               |district_id|province      |province_id|
+------------------------------------------------------------------------------+---------------------------------+---------+--------------------------+-------+-----------------------+-----------+--------------+-----------+
|Số 7, Phố Hàng Giấy, Phường Đồng Xuân, Hoàn Kiếm, Hà Nội                     |Số 7                             |-1       |Phố Hàng Giấy             |-1     |Phường Đồng Xuân      |-1         |Hoàn Kiếm     |-1         |
|Số 435+437, Phố Vũ Hữu, Phường Thanh Xuân Bắc, Thanh Xuân, Hà Nội             |Số 435+437                  

In [93]:
# Thống kê cơ bản về các cột số
df.describe().show()

# Kiểm tra phân phối giá trị của các cột dạng chuỗi (đếm tần suất các giá trị)
from pyspark.sql.functions import col

# Lặp qua các cột để kiểm tra phân phối giá trị của các cột dạng chuỗi
for column in df.columns:
    if dict(df.dtypes)[column] == 'string':  # Kiểm tra xem cột có phải là kiểu string không
        print(f"Phân phối giá trị cho cột: {column}")
        df.groupBy(column).count().show()


+-------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+---------+---------+---------+----------------+------------------+------------------+--------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------+-------------------------+-------------------+-------------+
|summary|                  id|                 url|    source|               title|         description|            location|data_type| province| district|            ward|          latitude|         longitude|               price|              area|       price_per_m2|           bedroom|          bathroom|      floor_count|  house_direction|      legal_status|          interior|        

In [94]:
# Kiểm tra các giá trị min và max cho các cột số
for column in df.columns:
    if dict(df.dtypes)[column] in ['double', 'int']:  # Kiểm tra chỉ với các cột số
        min_value = df.selectExpr(f"min({column})").collect()[0][0]
        max_value = df.selectExpr(f"max({column})").collect()[0][0]
        print(f"Cột: {column}")
        print(f"  Min: {min_value}")
        print(f"  Max: {max_value}")
        print("-" * 50)


Cột: latitude
  Min: 9.156794
  Max: 21.4526750773141
--------------------------------------------------
Cột: longitude
  Min: 103.9028
  Max: 109.372375
--------------------------------------------------
Cột: price
  Min: 2100000.0
  Max: 2100000000000.0
--------------------------------------------------
Cột: area
  Min: 10.0
  Max: 30000.0
--------------------------------------------------
Cột: price_per_m2
  Min: 100.0
  Max: 317780000000.0
--------------------------------------------------
Cột: bedroom
  Min: 1.0
  Max: 155.0
--------------------------------------------------
Cột: bathroom
  Min: 1.0
  Max: 100.0
--------------------------------------------------
Cột: floor_count
  Min: 1.0
  Max: 77.0
--------------------------------------------------
Cột: width
  Min: 1.0
  Max: 43000.0
--------------------------------------------------
Cột: length
  Min: 1.0
  Max: 14000.0
--------------------------------------------------
Cột: living_size
  Min: 2.0
  Max: 60100.0
-------------

In [103]:
# Lọc ra cột url và price
price_url_df = df.select("url", "price", "area","price_per_m2")

# Truy xuất các bản ghi có giá trị price bằng 1 (có thể là giá trị mặc định hoặc lỗi)
low_price_url_df = price_url_df.filter(col("price") < 50_000_000)
low_price_url_df.show(5, truncate=False)

# Truy xuất các bản ghi có giá trị price quá cao (> 200 tỷ)
high_price_url_df = price_url_df.filter(col("price") > 1_000_000_000_000)

high_price_url_df.show(5, truncate=False)
high_price_url_df.count()


+------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----+-----------------+
|url                                                                                                                                                         |price    |area |price_per_m2     |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----+-----------------+
|https://nha.chotot.com/125444794.htm                                                                                                                        |4.3E7    |98.0 |438775.5         |
|https://nha.chotot.com/124779035.htm                                                                                                                        |2100000.0|45.0 |46666.667        |
|https://nha.chotot.com/125411718.h

6

In [56]:
from pyspark.sql.functions import col, count, when

# Tính tổng số dòng trong DataFrame
total_rows = df.count()

# Kiểm tra nếu DataFrame có dữ liệu
if total_rows == 0:
    print("DataFrame rỗng, không có dữ liệu để kiểm tra.")
else:
    # Lấy danh sách các cột trong DataFrame
    columns = df.columns

    # Kiểm tra tỷ lệ thiếu trong từng cột
    missing_data = {}

    for column in columns:
        missing_count = df.filter(col(column).isNull()).count()
        missing_percentage = (missing_count / total_rows) * 100
        missing_data[column] = {"missing_count": missing_count, "missing_percentage": missing_percentage}

    # Hiển thị kết quả
    for column, data in missing_data.items():
        print(f"Cột: {column}")
        print(f"  Thiếu: {data['missing_count']} giá trị ({data['missing_percentage']:.2f}%)")
        print("-" * 50)


Cột: price
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: area
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: latitude
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: longitude
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: price_per_m2
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: bedroom
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: bathroom
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: district
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: ward
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: house_direction
  Thiếu: 0 giá trị (0.00%)
--------------------------------------------------
Cột: legal_status
  Thiếu: 0 giá trị (0.00%)
-----------------------------------

In [48]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType, DateType

# Chuyển cột dạng Date và Timestamp sang String
for name, dtype in df.dtypes:
    if dtype in ('timestamp', 'date'):
        df = df.withColumn(name, col(name).cast("string"))

df.printSchema()


root
 |-- price: double (nullable = true)
 |-- area: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- price_per_m2: double (nullable = true)
 |-- bedroom: double (nullable = true)
 |-- bathroom: double (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- house_direction: string (nullable = true)
 |-- legal_status: string (nullable = true)
 |-- interior: string (nullable = true)
 |-- house_type: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- data_quality_score: double (nullable = true)
 |-- id: string (nullable = true)
 |-- data_date: string (nullable = true)
 |-- bedroom_bathroom_ratio: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- area_per_room: double (nullable = true)
 |-- area_category: string (nullable = true)
 |-- price_vs_market: integer (nullable = true)
 |-- distance_to_center: double (nullable = true)
 |-- location_

In [51]:
from pyspark.sql.functions import col, count, when

# Tính tỷ lệ phần trăm của các giá trị thiếu cho mỗi cột
total_rows = df.count()

# Lấy danh sách các cột trong DataFrame
columns = df.columns

# Kiểm tra tỷ lệ thiếu trong từng cột
missing_data = {}

for column in columns:
    missing_count = df.filter(col(column).isNull()).count()
    missing_percentage = (missing_count / total_rows) * 100
    missing_data[column] = {"missing_count": missing_count, "missing_percentage": missing_percentage}

# Hiển thị kết quả
for column, data in missing_data.items():
    print(f"Cột: {column}")
    print(f"  Thiếu: {data['missing_count']} giá trị ({data['missing_percentage']:.2f}%)")
    print("-" * 50)


ZeroDivisionError: division by zero

In [41]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType, DateType

# Chuyển cột dạng Date và Timestamp sang String
for name, dtype in df.dtypes:
    if dtype in ('timestamp', 'date'):
        df = df.withColumn(name, col(name).cast("string"))

df.printSchema()
df.count()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)
 |-- data_type: string (nullable = true)
 |-- province: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- price: double (nullable = true)
 |-- area: double (nullable = true)
 |-- price_per_m2: double (nullable = true)
 |-- bedroom: double (nullable = true)
 |-- bathroom: double (nullable = true)
 |-- floor_count: double (nullable = true)
 |-- house_direction: string (nullable = true)
 |-- legal_status: string (nullable = true)
 |-- interior: string (nullable = true)
 |-- house_type: string (nullable = true)
 |-- width: double (nullable = true)
 |-- length: double (nullable = true)
 |-- living_size: double (nullable = true)


15344

In [34]:
import os

output_dir = "/tmp/house_data_csv"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

csv_output_path = f"{output_dir}/house_data.csv"

# Lưu DataFrame dưới dạng file CSV
df.toPandas().to_csv(csv_output_path, index=False)

# Kiểm tra file
print(f"File CSV đã được lưu tại: {csv_output_path}")
print(f"Kích thước file: {os.path.getsize(csv_output_path) / (1024*1024):.2f} MB")


File CSV đã được lưu tại: /tmp/house_data_csv/house_data.csv
Kích thước file: 84.88 MB


In [36]:
from IPython.display import HTML
import base64

def create_download_link(file_path, filename=None):
    if filename is None:
        filename = os.path.basename(file_path)

    with open(file_path, "rb") as f:
        data = f.read()

    b64 = base64.b64encode(data).decode()
    href = f'<a download="{filename}" href="data:text/csv;base64,{b64}" target="_blank">Tải xuống {filename}</a>'
    return HTML(href)

# Hiển thị link tải xuống
create_download_link(csv_output_path, "house_data.csv")

/tmp/data đã được xóa.


In [1]:
spark.stop()

NameError: name 'spark' is not defined